In [1]:
import pandas as pd

In [2]:
import numpy as np

In [13]:
taxies = pd.read_csv("taxi_prepared.csv", index_col=0)

In [15]:
taxies.head()

,vendor_id,pickup_datetime,store_and_fwd_flag,trip_duration,distance_km,category_encoded
id,,,,,,
id2875421,1,2016-03-14 17:24:55,0,455.0,1.498521,930.399753
id2377394,0,2016-06-12 00:43:35,0,663.0,1.805507,930.399753
id3858529,1,2016-01-19 11:35:24,0,2124.0,6.385098,930.399753
id3504673,1,2016-04-06 19:32:31,0,429.0,1.485498,930.399753
id2181028,1,2016-03-26 13:30:55,0,435.0,1.188588,930.399753


Попробуем построить модель Линейной Регрессии.

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pikcup_datetime*).

Для начала воспользуемся "коробочным решением". Создадим переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**.

Массив **X**, состоящий из признаков объектов.

In [66]:
X = taxies.drop(['pickup_datetime', 'trip_duration'], axis=1)

In [67]:
X

,vendor_id,store_and_fwd_flag,distance_km,category_encoded
id,,,,
id2875421,1,0,1.498521,930.399753
id2377394,0,0,1.805507,930.399753
id3858529,1,0,6.385098,930.399753
id3504673,1,0,1.485498,930.399753
id2181028,1,0,1.188588,930.399753
...,...,...,...,...
id2376096,1,0,1.225080,1053.529749
id1049543,0,0,6.049836,930.399753
id2304944,1,0,7.824606,930.399753


Mассив **Y** с таргетами.

In [68]:
Y = taxies['trip_duration']

In [95]:
Y

id
id2875421     455.0
id2377394     663.0
id3858529    2124.0
id3504673     429.0
id2181028     435.0
              ...  
id2376096     778.0
id1049543     655.0
id2304944     764.0
id2714485     373.0
id1209952     198.0
Name: trip_duration, Length: 1458644, dtype: float64

Посмотрим кoэффициенты модели

In [96]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X, Y)

LinearRegression()

In [97]:
model.coef_

array([198.46622125,  56.61095594, 115.26449772,   0.29633374])

Посмотрим свободный вес модели.

In [98]:
model.intercept_

172.07109507020095

Теперь реализуем функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуем решение через аналитическое решение в матрицах:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [143]:
def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True) -> np.array:
    if fit_intercept == True:
      X = np.column_stack((X, np.ones(X.shape[0])))
    return np.linalg.inv(X.T @ X) @ X.T @ Y

In [147]:
betas = LinearRegressionByMatrix(X, Y, False) 

In [148]:
betas

0    194.862223
1     57.176574
2    115.310518
3      0.477083
dtype: float64

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта. Это можно сделать через операцию матричного произведения матрицы **X** и полученных коэффициентов.


In [150]:
predictions = np.dot(X, betas)

In [151]:
predictions

array([ 811.53575238,  652.07228872, 1375.00955879, ..., 1540.99988857,
        569.86243031,  574.64528803])